# **Install the Necessary Tools and Libaraies**
We'll use:
1. HuggingFace Transformers:For pre-trained language models like`T5`, `GPT`, or `DistilBERT`.
2. FAISS/Dense Retrieval:For building the retrieval component(to fetch relevant knowledge with data tables).
3. LangChain:For combining retrieval and geeration effectively.
4. Flask/FastAPI:To create a RESTfull API for Chatbot.
5. Libraries for Preprocessing and Vectorization:
   - `pandas` for handling tables.
   - `sentence-transformers` for vector embeddings.

In [1]:
%%time
!pip install -q faiss-cpu sentence-transformers
import pandas as pd
import sqlite3
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline

CPU times: user 11.7 s, sys: 2.27 s, total: 14 s
Wall time: 32.7 s


# **Load the Dataset from `database.db`, Extract Tables, Preprocess Data, and Generate Embeddings**

This step involves:
1. Loading the dataset from a SQLite database (`database.db`).
2. Extracting tables (`faqs` and `menu_items`) into `pandas` DataFrames.
3. Preprocessing the data to clean and standardize it.
4. Generating embeddings for each record to enable retrieval.

---

### **Steps to Implement**

#### 1. **Connect to SQLite Database and Load Tables**
We use the `sqlite3` library to connect to the database and extract tables.

#### 2. **Preprocess the Data**
- **FAQs Table**:
  - Standardize questions and answers (e.g., remove duplicates or trim spaces).
  - Combine question-answer pairs for embedding generation.
- **Menu Items Table**:
  - Combine menu item details (name, description, ingredients, allergens) for embeddings.
  - Handle missing values or incorrect formats.

#### 3. **Generate Embeddings**
- Use the `sentence-transformers` library to generate embeddings for text fields.
- Store the embeddings in memory or save them for later use.

In [2]:
%%time
# Step 1: Connect to SQLite database and extract tables
db_path = "/kaggle/input/dataset-resturent-ksa/restaurant.db"
conn = sqlite3.connect(db_path)

# Load tables into DataFrames
faqs_df = pd.read_sql_query("SELECT * FROM faqs;", conn)
menu_items_df = pd.read_sql_query("SELECT * FROM menu_items;", conn)

# Close the database connection
conn.close()

CPU times: user 9.77 ms, sys: 346 µs, total: 10.1 ms
Wall time: 39.7 ms


In [3]:
%%time
# Step 2: Preprocess Data
# FAQs Table Preprocessing
faqs_df = faqs_df.drop_duplicates()  # Remove duplicate rows
faqs_df['combined'] = faqs_df['question'] + " " + faqs_df['answer']  # Combine question and answer for embeddings

# Menu Items Table Preprocessing
menu_items_df['combined'] = (
    menu_items_df['name'] + " " +
    menu_items_df['description'] + " Ingredients: " +
    menu_items_df['ingredients'] + " Allergens: " +
    menu_items_df['allergens']
)

CPU times: user 5.16 ms, sys: 231 µs, total: 5.39 ms
Wall time: 9.12 ms


In [4]:
%%time
# Handle missing or null values
faqs_df.fillna("", inplace=True)
menu_items_df.fillna("", inplace=True)

CPU times: user 828 µs, sys: 0 ns, total: 828 µs
Wall time: 814 µs


# **Generate Embeddings** `Initialize SentenceTransformer model`

In [5]:
%%time
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 553 ms, sys: 371 ms, total: 925 ms
Wall time: 3.97 s


## **Generate embeddings for FAQs**

In [6]:
%%time
# Generate embeddings for FAQs
faqs_df['embedding'] = faqs_df['combined'].apply(lambda x: embedding_model.encode(x))
faqs_df.head()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 628 ms, sys: 219 ms, total: 847 ms
Wall time: 999 ms


,id,question,answer,combined,embedding
0,1,What are your restaurant's hours of operation?,Our restaurant is open from 12:00 PM to 11:00 ...,What are your restaurant's hours of operation?...,"[0.020295117, -0.044048678, 0.007860406, 0.065..."
1,2,What are your restaurant's hours of operation?,Our restaurant is open from 12:00 PM to 11:00 PM.,What are your restaurant's hours of operation?...,"[0.023397474, -0.02871275, 0.00077521335, 0.04..."
2,3,Do you offer vegetarian options?,"Yes, we have a variety of vegetarian dishes av...","Do you offer vegetarian options? Yes, we have ...","[0.03610269, -0.06566538, 0.013662725, 0.06345..."
3,4,What is the best dish on your menu?,Our signature dish is the grilled salmon with ...,What is the best dish on your menu? Our signat...,"[-0.039106302, -0.013256464, 0.019794157, 0.00..."
4,5,Do you have a kids' menu?,"Yes, we offer a kids' menu with smaller portio...","Do you have a kids' menu? Yes, we offer a kids...","[0.033577904, 0.025054036, 0.051779628, 0.0381..."


# **Generate embeddings for Menu Items**

In [7]:
%%time
menu_items_df['embedding'] = menu_items_df['combined'].apply(lambda x: embedding_model.encode(x))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 22 s, sys: 2.32 s, total: 24.3 s
Wall time: 21.9 s


## **Save Embeddings for `FAQs &  Menu Items`**

In [8]:
%%time
faqs_df.to_pickle("faqs_embeddings.pkl")
menu_items_df.to_pickle("menu_items_embeddings.pkl")

CPU times: user 7.53 ms, sys: 5.3 ms, total: 12.8 ms
Wall time: 11.7 ms


## **Disply preprocessed and embedded data**

In [9]:
print("FAQs DataFrame with Embeddings:")
print(faqs_df.head())
print("\nMenu Items DataFrame with Embeddings:")
print(menu_items_df.head())

FAQs DataFrame with Embeddings:
   id                                        question  \
0   1  What are your restaurant's hours of operation?   
1   2  What are your restaurant's hours of operation?   
2   3                Do you offer vegetarian options?   
3   4             What is the best dish on your menu?   
4   5                       Do you have a kids' menu?   

                                              answer  \
0  Our restaurant is open from 12:00 PM to 11:00 ...   
1  Our restaurant is open from 12:00 PM to 11:00 PM.   
2  Yes, we have a variety of vegetarian dishes av...   
3  Our signature dish is the grilled salmon with ...   
4  Yes, we offer a kids' menu with smaller portio...   

                                            combined  \
0  What are your restaurant's hours of operation?...   
1  What are your restaurant's hours of operation?...   
2  Do you offer vegetarian options? Yes, we have ...   
3  What is the best dish on your menu? Our signat...   
4  Do yo

# **Build the Retrieval System**
In this step, we'll:
 1. Create a retrieval system for the FAQ and menu data using the embeddings we generated.
 2. Use `FAUSS`(Facebook AI Similarity Search) to build a vector store for efficient retrieval of similar text based on embeddings.

### Step 1: Prepare the Embeddings for Indexing
### Convert embeddings for DataFrame to NumPy arrays

In [10]:
%%time
faq_embeddings = np.vstack(faqs_df['embedding'].values)
menu_embeddings = np.vstack(menu_items_df['embedding'].values)

CPU times: user 2 ms, sys: 1.01 ms, total: 3.01 ms
Wall time: 2.75 ms


### Step 2: Create FAISS Index for FAQs

In [11]:
%%time
faq_index = faiss.IndexFlatL2(faq_embeddings.shape[1])
faq_index.add(faq_embeddings)

CPU times: user 190 µs, sys: 30 µs, total: 220 µs
Wall time: 287 µs


### Step 3: Create FAISS Index for Menu Items

In [12]:
%%time
menu_index = faiss.IndexFlatL2(menu_embeddings.shape[1])  # L2 similarity
menu_index.add(menu_embeddings)  

CPU times: user 905 µs, sys: 841 µs, total: 1.75 ms
Wall time: 1.45 ms


## **Step Four: `Define a Retrieval Funcation`**

In [13]:
def retrieve(query, model, faq_index, menu_index, top_k=3):
    """
    Retrieve top-k similar entries from the FAQ and Menu indices.
    
    Args:
        query (str): User input query.
        model: SentenceTransformer model for encoding.
        faq_index: FAISS index for FAQ data.
        menu_index: FAISS index for Menu Items data.
        top_k (int): Number of top results to return.
    
    Returns:
        dict: Retrieved results from FAQs and menu items.
    """
    # Encode the query
    query_embedding = model.encode(query).reshape(1, -1)

    # Retrieve from FAQ index
    faq_distances, faq_indices = faq_index.search(query_embedding, top_k)
    faq_results = faqs_df.iloc[faq_indices[0]].to_dict(orient="records")

    # Retrieve from Menu index
    menu_distances, menu_indices = menu_index.search(query_embedding, top_k)
    menu_results = menu_items_df.iloc[menu_indices[0]].to_dict(orient="records")

    return {
        "query": query,
        "faq_results": faq_results,
        "menu_results": menu_results
    }

### **Example Usage**

In [14]:
# Example Usage
query = "What are your restaurant's hours of operation?"
results = retrieve(query, embedding_model, faq_index, menu_index)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
# # Display Retrieved Results
# print("Query:", results["query"])
# print("\nTop FAQ Results:")
# for res in results["faq_results"]:
#     print(res)

# print("\nTop Menu Results:")
# for res in results["menu_results"]:
#     print(res)

# ** Integrate a Response Generator**
## **Overview of the Response Generator**
1. Retrieve Relevant knowledge:
   - Use the retrieval system to fetch relevant FAQs or Menu Items based on the user query.
2. Generate Responses:
   - Use a pre-trained language model like `GPT-3`, `Flan-T5, or any HuggingFace conversational model to create user-friendly responses.
   - Combine the user query with retrieved knowledge for context.
# **Implementation: Retrieval-Augmented Response Generation**

In [16]:
%%time
# Step 1: Initialize a Text Generation Model
# Use a pre-trained model from HuggingFace (e.g., Flan-T5 for Q&A or conversational tasks)
generator = pipeline("text2text-generation", model="google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


CPU times: user 1.82 s, sys: 1.72 s, total: 3.54 s
Wall time: 6.53 s


### **Define Funcation for Respons Generation**

In [38]:
def generate_response(query, retriever, model, faq_index, menu_index, top_k=3):
    """
    Generate a chatbot response that addresses both FAQ and menu item queries effectively.
    
    Args:
        query (str): User input query.
        retriever: Retrieval function for fetching relevant data.
        model: HuggingFace pipeline for generating text.
        faq_index: FAISS index for FAQ data.
        menu_index: FAISS index for menu items data.
        top_k (int): Number of top results to retrieve.
    
    Returns:
        str: Generated chatbot response.
    """
    # Step 1: Retrieve Relevant Data
    retrieved_data = retriever(query, embedding_model, faq_index, menu_index, top_k)
    faq_results = retrieved_data['faq_results']
    menu_results = retrieved_data['menu_results']


    # Step 3: Prepare Context for Generation
    context = "You are a helpful assistant for a restaurant in Saudi Arabia. Answer the question based on the provided context.:\n\n"

    # Add FAQ context if the query is FAQ-related
    context += "FAQs:\n"
    for faq in faq_results:
        context += f"- Q: {faq['question']} A: {faq['answer']}\n"

    context += "\nMenu Items:\n"
    for item in menu_results:
        context += f"- {item['name']}: {item['description']} (Ingredients: {item['ingredients']}, Allergens: {item['allergens']})\n"

    # Ensure the user query is included in the prompt
    context += f"\nUser Query: {query}\n\n"

    # print(context)
    # Step 4: Generate a Response
    response = generator(
        context,
        max_length=200,
        do_sample=True,       # Enable sampling
        temperature=0.4,      # Set temperature for balanced randomness
        top_p=0.5,            # Use nucleus sampling
        top_k=50,             # Restrict to top 50 tokens
        num_return_sequences=1)

    return response[0]['generated_text']

## **Exampl Usage**

In [40]:
%%time
user_query = "Do you have a kids' menu?"
response = generate_response(user_query, retrieve, generator, faq_index, menu_index)

# Display the Generated Response
print("User Query:", user_query)
print("\nChatbot Response:")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

User Query: Do you have a kids' menu?

Chatbot Response:
Yes, we offer a kids' menu with smaller portions and kid-friendly options.
CPU times: user 2.9 s, sys: 24.3 ms, total: 2.93 s
Wall time: 1.48 s


# **Set Up Google Cloud Translation API**